In [3]:
import torch
import numpy as np
import os
import gpt as g
import matplotlib.pyplot as plt

import sys
sys.path.append("src/")
from qcd_ml.dirac import dirac_wilson_clover
from qcd_ml.nn.lptc import v_LPTC_NG
from qcd_ml.nn.ptc import v_PTC
from qcd_ml.dirac import dirac_wilson_clover

from stuff import *

In [2]:
class Smoother_PTC(torch.nn.Module):
    def __init__(self, U):
        super().__init__()
        self.U = U
        self.paths = [[]] + [[(mu, 1)] for mu in range(4)] + [[(mu, -1)] for mu in range(4)]
        self.l0 = v_PTC(2, 2, self.paths, self.U)
        self.l1 = v_PTC(2, 2, self.paths, self.U)
        self.l2 = v_PTC(2, 2, self.paths, self.U)
        self.l3 = v_PTC(2, 1, self.paths, self.U)

    def forward(self, v):
        for l in [self.l0, self.l1, self.l2, self.l3]:
            v = l.forward(v)
        return v

In [4]:
class Multigrid_model(torch.nn.Module):
    def __init__(self, U, mg_setup):
        self.mg_setup = tuple(mg_setup)
        self.coarse_model = v_LPTC_NG
        self.smoother = Smoother_PTC(U)

    def forward(self, v):
        v1 = v
        v2 = torch.clone(v)
        v2_c = v_project(*self.mg_setup, v2)
        v2_c = self.coarse_model.forward(v2_c)
        v2 = v_prolong(*self.mg_setup, v2_c)
        return self.smoother.forward(v1, v2)

In [5]:
mg_setup = torch.load("mg_setup.pt")
smoother_weights = torch.load("smoother_start_id.pt")
coarse_weights = torch.load("coarse.pt")

In [ ]:
vec = torch.complex(
        torch.randn(8, 8, 8, 16, 4, 3, dtype=torch.double)
        , torch.randn(8, 8, 8, 16, 4, 3, dtype=torch.double))

U = torch.tensor(np.load(os.path.join("test", "assets","1500.config.npy")))